## Conserving models 

- save weights and shape of the network so that it can be started quickly

# make it with vgg

In [1]:
#import modules
import utils; reload(utils)
from vgg16 import Vgg16

#Instantiate plotting tool
#In Jupyter notebooks, you will need to run this command before doing any plotting
%matplotlib inline

DATA_HOME_DIR = './data/dogsandcats'

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
path = DATA_HOME_DIR + '/' #'/sample/'
test_path = DATA_HOME_DIR + '/test/' #We use all the test data
results_path=DATA_HOME_DIR + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'

In [3]:
%mkdir -p ./data/dogsandcats/results

## instantiate the vgg16 class

In [4]:
vgg = Vgg16()

In [5]:
#Set constants. You can experiment with no_of_epochs to improve the model
batch_size=64
no_of_epochs=15

In [6]:
#Finetune the model
batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size*2)
vgg.finetune(batches)

#Not sure if we set this for all fits


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [8]:
#Notice we are passing in the validation dataset to the fit() method
#For each epoch we test our model against the validation set
latest_weights_filename = None

# time the test
import time
start_time = time.time()


for epoch in range(no_of_epochs):
    print("Running epoch: %d" % epoch)
    vgg.fit(batches, val_batches, nb_epoch=1)
    latest_weights_filename = 'ft%d.h5' % epoch
    vgg.model.save_weights(results_path+latest_weights_filename)
         

elasped = time.time() - start_time
print( " TOTAL TIME for %d epochs: %d" % (no_of_epochs, elasped) )
print( "Completed %s fit operations" % no_of_epochs)



Running epoch: 0
Epoch 1/1
20000/20000 [==============================] - 592s - loss: 0.1091 - acc: 0.9829 - val_loss: 0.1065 - val_acc: 0.9838
Running epoch: 1
Epoch 1/1
20000/20000 [==============================] - 592s - loss: 0.1123 - acc: 0.9817 - val_loss: 0.0852 - val_acc: 0.9856
Running epoch: 2
Epoch 1/1
17792/20000 [=========================>....] - ETA: 52s - loss: 0.1151 - acc: 0.9821

KeyboardInterrupt: 

### Prediction and Kaggle submission


### Use already trained models

can determine improvement over successive epochs

In [4]:
# just what we need in vgg
import utils; reload(utils); from vgg16 import Vgg16; import time
from utils import *

vgg = Vgg16()

batch_size=64
no_of_epochs=3

batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size*2)
vgg.finetune(batches)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [10]:
%ls ./data/dogsandcats/results
%mkdir -p ./data/dogsandcats/test/unknown

filenames.dat/  ft11.h5  ft14.h5  ft3.h5  ft6.h5  ft9.h5
ft0.h5          ft12.h5  ft1.h5   ft4.h5  ft7.h5  test_preds.dat/
ft10.h5         ft13.h5  ft2.h5   ft5.h5  ft8.h5


In [5]:
## use an old model

vgg.load_model('./data/dogsandcats/results/', 'ft14.h5')

In [6]:
%ls data/dogsandcats/test/unknown/ | wc -l


12500


In [9]:
# predict and time the prediction
start_time = time.time()



batches, preds = vgg.test(test_path)


elasped = time.time() - start_time

print( " TOTAL TIME inference %d mins" %  (elasped / 60) )
print( " %d inferred per sec" %  (12500 / elasped) )


Found 12500 images belonging to 1 classes.
 TOTAL TIME inference 5 mins
 39 inferred per sec


## Save prediction array

In [69]:
#Save our test results arrays so we can use them again later
save_array(results_path + 'test_preds.dat', preds)
save_array(results_path + 'filenames.dat', batches.filenames)

In [12]:
# load old preditions
preds = load_array(results_path + 'test_preds.dat')
filenames = load_array(results_path + 'filenames.dat')

## Exploring the prediction data 
plots and plots and plots

In [32]:
# print an image
print filenames[:5]

preds[:,0]

['unknown/9292.jpg' 'unknown/12026.jpg' 'unknown/9688.jpg' 'unknown/4392.jpg' 'unknown/779.jpg']


array([  1.0000e+00,   8.9735e-01,   1.0000e+00, ...,   1.0000e+00,   1.0000e+00,   1.2811e-10], dtype=float32)

In [ ]:
from keras.preprocessing import image



### some space









 df
 

In [19]:
# format predictions into a dataframe

ids = map(lambda img: int(img.split('.')[0]), test_batches.class_indices)
dogs = map(lambda arr: arr[1], preds)
    
# clip . predictions
probablity_is_dog = map(lambda pred: 0.90000 if pred > 0.90001 else pred  , dogs )
# probablity_is_dog = dogs


NameError: name 'preds' is not defined

In [ ]:
df = pd.DataFrame({
    'id' : ids,
    'label' : probablity_is_dog
 })

# sort by image id number
df = df.sort_values(by=['id'])

print(df.head(10))

In [ ]:
#  WRITE to CSV
submission = open('./submission.csv', 'a+')
submission.write('id,label\n')

for index, row in df.iterrows():
    string = "{:.0f},{:.10f}\n".format(row['id'], row['label'])
#     print(string)
    submission.write(string)

    
submission.close()